In [ ]:
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import *
import os
import pickle

# Convert CSV data to text file

In [ ]:
# Read Data
CSV_Data = pd.read_csv('@Data/Music Lyrics/Lyrics.csv', encoding = "ISO-8859-1")
TXT_Data = '@Data/Music Lyrics/Lyrics.txt'

# Drop NA row
CSV_Data['Lyrics'].replace(['NA'], [np.nan], inplace=True)
CSV_Data.dropna(axis=0, how='any', inplace=True)

# Save Data
CSV_Data.to_csv('@Data/Music Lyrics/Final.csv', encoding="utf8",  index = False)
Final_Data = '@Data/Music Lyrics/Final.csv'

with open(TXT_Data, "w") as my_output_file:
    with open(Final_Data, "r", encoding="utf8") as my_input_file:
        [ my_output_file.write(" ".join(row) + ' ') for row in csv.reader(my_input_file)]
    my_output_file.close()

# Initiate Variables

In [ ]:
maxlen = 25
char_idx_file = '@Data/Music Lyrics/char_idx.pickle'

char_idx = None
if os.path.isfile(char_idx_file):
    print('Loading previous char_idx')
    char_idx = pickle.load(open(char_idx_file, 'rb'))

In [ ]:
X, Y, char_idx = textfile_to_semi_redundant_sequences(TXT_Data, seq_maxlen=maxlen, redun_step=3, pre_defined_char_idx=char_idx)
pickle.dump(char_idx, open(char_idx_file,'wb'))

# LSTM

In [ ]:
g = tflearn.input_data([None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001)

In [ ]:
m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='model_music')

In [ ]:
for i in range(50):
    seed = random_sequence_from_textfile(TXT_Data, maxlen)
    m.fit(X, Y, validation_set=0.1, batch_size=128,
          n_epoch=1, run_id='music')
    print("-- TESTING...")
    print("-- Test with temperature of 1.0 --")
    print(m.generate(600, temperature=1.0, seq_seed=seed))
    print("-- Test with temperature of 0.5 --")
    print(m.generate(600, temperature=0.5, seq_seed=seed))